<a href="https://colab.research.google.com/github/shahnazumer/Qdrant/blob/main/qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain qdrant_client openai tiktoken

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
import qdrant_client
import os

In [ ]:
# create qdrant client

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [ ]:
# create collection

os.environ["QDRANT_COLLECTION_NAME"]

vectors_config = qdrant_client.http.models.VectorParams(
    size=1536, # 768 for instructor-xl, 1536 for OpenAI
    distance=qdrant_client.http.models.Distance.COSINE
  )

client.create_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)

True

In [ ]:
# create vector store

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings=embeddings,
)

In [ ]:
# add document to vector store

from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )

  chunks = text_splitter.split_text(text)
  return chunks

with open('story.txt') as f:
  raw_text = f.read()

texts = get_chunks(raw_text)

vector_store.add_texts(texts)

['e6fd86a8d88e4154b2055d8d72b5fa16', 'ac0289e0edcb431a9c3f7349865079b2']

In [ ]:
from langchain.schema import retriever
#plug vector store into retrieval chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",  # Positional argument before keyword argument
    retriever=vector_store.as_retriever()
)

In [ ]:
query = "What is Dream of the Red Chamber known as?"
response = qa.run(query)

print(response)

 Dream of the Red Chamber is also known as "The Story of the Stone."
